# Pseudo-code for MLP from scratch
by the ML2021-2 TA 

Hello! Please read this document carefully. I hope it helps understand how MLP for a classification task works, and a general idea of how a neural network can 'learn'. I will start form the very basic intuition, and then build up from there until the actual implementation of the MLP.

First, we will start from the task. We want to classify some images from the MNIST dataset. That is, we want a model that takes an image as an iput and reutrns the label.

<div>
<img src=https://drive.google.com/uc?export=view&id=1KZzE6S5bs0p6IL0xZXX4FDInvmo_2Sfw width="500"/>
</div>


Basically speaking, this is just a (veeeeery complex) function y = f(x), where f is replaced with the model.

In our case, f(.)(the model) is the MLP network.



## The input

The input to our model would be the images. As we have seen many times, this images are described by their pixel values. For the MNIST case (grey-scale 28x28 pixel images), we only need one value for each pixel, ranging between 0-255.

Basically, our model should learn how each pixel value is related to the label of the images. The intuition is answering the question 'Why do this 784 pixel values represent a number 4?', for example.

First step, we want to 'flatten' our images. Instead of an array (matrix) of 28x28, we will input a 784 vector. That is, we will make a vector by puting each pixel of the image one next to the other. For the MLP, this step is a computational convenience. For an illustrative example, refer to the image below:

<div>
<img src=https://drive.google.com/uc?export=view&id=1lDznKUeVk-3neVJAC0qCchFFB6ZykFkp width="300"/>
</div>




##Supervised learning and overall training

This classification task is using supervised learning. This means that during training we can adjust our estimation according to the correct labels.

The intuitive idea is as follows:
Given all the input images, our model will precdict their labels and store the 'reasoning' that led to this classification decision in the form of parameters (W, b). Then, we will compute a cost fucntion. This cost function is a summation of all the errors in our predictions w.r.t the true labels. Intuitively, we want this cost function to be as small as possible, since it will mean that we made little wrong predictions. The process described in this paragraph is called ***forward propagation***.

But what if the classification that the model made is wrong for some labels? Given the cost function, we want to keep the 'reasoning' that led to a correct label, but change the 'resoning' that led to an incorrect label in such a way that next time, when we see that image, we will predict the correct label. To do so, we have to 'go back' in our reasoning process starting from this cost function and change the parameters that led to a wrong prediction. We do this via computing the gradients of all the variables and parameters that led to this cost function. This process is called ***backward propagation***. 

Finally, with the information of the gradients, we want to change the parameters a little in the direction that minimizes the cost function. There are several ways of updating the parameters, and this step is called optimization. Finally, after we change our parameters, we update the old ones by replacing them with these changed ones. It is like updating our 'reasoning'. 

After all this, we forward propagate all the images one more time with the new parameters and obtain a new classification. We compare our estimation with the true labels and backward propagate. We update our parameters. And repeat. 

To recapitulate, training an MLP classifier has three main parts:

1. **Forward propagation:** classify all the images in the training set using the model and check the the predicted labels with the true labels using the cost function.
2. **Backward propagation:** go back in our reasoning and compute the derivatives of our cost function w.r.t. the parameters and variables we computed.
3. **Update:** change the parameter values that gave us the wrong labels in the direction that minimizes the error. Update the parameter values for the next forward propagation.

Finally, when we trained our model to make the best possible predictions, we stop and save these parameters.

To test if our model can generalize, we apply forward propagation to a new dataset (called test dataset) and we check the accuracy of our label predictions in this set. Why don't we backpropagate in this case? Because we are done training, we don't want to update the parameters any more with this test set (that is why is called test set ^^).

## MLP architecture

Now we will talk about the MPL (our model) architecture. 
The MLP architecture is made of:

1.   **Input:** The input vector to our model (in this case, 784-dimensional vectors)
2.   **Hidden Layer(s):** a layer is a set of *k* neurons. Each neuron will capture information about each input pixel and store it in the parameters. We can have different layers with different number of neurons.
3. **Output Layer:** The last neuron that outputs the prediction (in this case, the labels)

In this case, Professor said that we need only one hidden layer. So, our architecture will be like this: 

<div>
<img src=https://drive.google.com/uc?export=view&id=1EmTaFAmiFU3JAmkLddoyTvezbTjwM8kt width="500"/>
</div>

---

**How many neurons should this hidden layer have?**

The answer is: you have to choose. There is no rule to determine a correct number of neurons. 
A common question is if this nerons should be the same number as the input image. They could be, but it is not mandatory. However, is it convenient? If we have too many neurons, we might capture too much useless information. But if we have too little amount, then we might fail to capture some details. The best strategy is to try different number of neurons. 

##Let's code!
Now we are going to see each part in detail and also I will show a pseudo code for each one. If you understand correctly every step, you will be able to fill this colab. with the correct values

In [1]:
#### Import your libraries (remember, no built-in library)
import six.moves.cPickle as pickle
import gzip
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import random
from sklearn import metrics
import time

Function for loading the data

In [ ]:
def load_data(dataset):
    ''' Loads the dataset

    :type dataset: string
    :param dataset: the path to the dataset (here MNIST)
    
    copied from http://deeplearning.net/ and revised by hchoi
    '''

    # Download the MNIST dataset if it is not present
    data_dir, data_file = os.path.split(dataset)
    if data_dir == "" and not os.path.isfile(dataset):
        # Check if dataset is in the data directory.
        new_path = os.path.join(
            os.path.split(__file__)[0],
            dataset
        )
        if os.path.isfile(new_path) or data_file == 'mnist.pkl.gz':
            dataset = new_path

    if (not os.path.isfile(dataset)) and data_file == 'mnist.pkl.gz':
        from six.moves import urllib
        origin = (
            'http://www.iro.umontreal.ca/~lisa/deep/data/mnist/mnist.pkl.gz'
        )
        print('Downloading data from %s' % origin)
        urllib.request.urlretrieve(origin, dataset)

    print('... loading data')

    # Load the dataset
    with gzip.open(dataset, 'rb') as f:
        try:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        except:
            train_set, valid_set, test_set = pickle.load(f)
    
    return train_set, valid_set, test_set


In [ ]:
  train_set, val_set, test_set = load_data('./mnist.pkl.gz')
    
  ## I will not use a validation set, but you can if you want to :)
  ## train_x and test_x contain the images for training and testing. 
  train_x, train_y = train_set
  test_x, test_y = test_set
  print('The amount of images in your training set is: ', train_x.shape[0])
  print('The amount of images in your testing set is: ', test_x.shape[0])

... loading data
The amount of images in your training set is:  50000
The amount of images in your testing set is:  10000


## Pre-processing
Now, we have to pre-process the dataset. The first one is to scale our dataset by substracting the mean. This is called normalization and it helps the model to have inputs that have a similar scale of values.

In [ ]:
def norm(X):
    return X - np.mean(X, axis=0)
    
train_scaled = norm(train_x)
test_scaled = norm(test_x)

Now, we will define a function to convert our labels into one-hot-encoded vectors. This is also convenient in terms of computation during our training. Instead of having the labels as integers (0,1,2,3,4...,9) we will write them as 10 dimensional vectors of 0s and 1s as follows:

$0 = [1,0,0,0,0,0,0,0,0,0]$

$1 = [0,1,0,0,0,0,0,0,0,0]$

$2 = [0,0,1,0,0,0,0,0,0,0]$

.
.
.

$9 = [0,0,0,0,0,0,0,0,0,1]$

Lucky for us, the libraries like pytorch and similar do this automatically for us, so this step is not necessary when we use them. However, for this part of the homework we have to do this manually (ㅠㅠ)

In [ ]:
##We will use it later during training :)
def one_hot(y):
    one_hot = []
    oh = np.zeros(10, dtype=int)
    for i in y:
        oh[i] = 1
        one_hot.append(oh)
        oh = np.zeros(10, dtype=int)
    return np.asarray(one_hot)

##One-hot encode for all the labels:
##Y = one_hot(train_y)
##print(Y, Y.shape)
##print('\n')
##Let's try with one example!
##print('If the integer label is: ', train_y[2])
##print('its one-hot-vector is: ', Y[2])

## Model hyperparameters

Now we are going to define the model hyperparameters and other needed values. Remember, hyperparamenters mean **non**-trainable parameters, so they will not change throughout our training.

In [ ]:
#Input dimension
x_dim = 28 * 28 

#Number of neurons in our only hidden layer
h = 200 

#Number of classes (10 classes for us)
C = 10 

#Learning rate (how much do we want to change the parameters when we update, see details below):
lr = 0.01

#Epochs (How many times do we want to forward and backpropagate all our data?) 
#Do not decrease this value (I tried this code and it needs a lot of epochs to train), but you can increase it if you want.
num_epochs=10000

## Initialization

First, we need to define and initialize our parameters from the hidden layer and the ouput layer. We have 2 sets of parameter matrices ($W_{xh}$, $b_h$) for our only hidden layer and ($W_o$, $b_o$) for the output layer. 

Can you think of the shape of these before we compute them? As a hint, refer to the structure of MLP:

<div>
<img src=https://drive.google.com/uc?export=view&id=1Is995KDQ486LkOHFi-R8A1RYmlYoKqCE width="500"/>
</div>

In [ ]:
def initp(x_dim, h, C):

    ##Initialize each array according to the corresponding dimension
    ## W@ are inistialized randomly, and b@ can be initialized as zero vectors (why can't W@ be zero?)

    W_xh = np.random.randn(x_dim, ...) #complete the  ...
    b_h = np.zeros((1, ...))
    Wo = np.random.randn(h, ...) 
    bo = np.zeros((1, ...))
    
    ##Save the parameter values as a dictionary. 
    ##This is our 'memory' where we store our parameters, and later we update them here.
    parameters = {"W_xh": W_xh,
                  "b_h": b_h,
                  "Wo": Wo,
                  "bo": bo}
    
    return parameters

#parameters = initp(x_dim, h, C) ## <- check the dimensions are correct!

## Forward Propagation

Now let's see forward propagation in more detail. From the slides in the ppt (slide 10), each layer will compute $y_t$ and an activation $h_t$. Pay attention to the subscripts. ALSO, I changed the notation a little bit to make it a little more clear, but the equations are the same as in the ppt.

Each neuron in the hidden layer will take each pixel value from the input $X$ and multiply it by the weight matrix $W_{xh}$ and add the bias vector $b_h$. If we use matrix notation (so we don't have to use a for loop), this will be:

$Z_1=W_{xh} * X + b_h$

Then, we need to add the non-linearity through an activation function. See the pdf slides for some examples of activation functions. Again there is no rule to choose activation functions. For example, if we choose ReLU:

$A_1=ReLU(Z_1)$

This is it for the hidden layer. The input are the images $X$ as an array. The output is an array $A_1$.

Now, this $A_1$ is the array that contains information about the image. This is the input to our next layer (which in our case is our final layer, the output layer.) In this layer we use $A_1$ to compute a new vector 

$Z_o=W_{ho} * A_1 + b_o$

And now, we transform this vector into probabilities using the softmax function:

$A_o=softmax(Z_o)$

That's it! This vector will give the probabilities of the input image being a certain label (so ten probabilities for each image in our array $X$... now you see why we needed the one-hot encoded vector? ^^)

In [ ]:
##Let's do everything from scratch! even the softmax function :)
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) 

def forward_prop(X, parameters):
    
    ##Let's load our current parameters
    W_xh = parameters["W_xh"]
    b_h = parameters["b_h"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    
    ##This is basically the MLP:
    ##hidden layer:
    Z1 = np.dot(W_xh.T, X) + b_h.T
    A1 = your_act(Z1) ##replace your_act with an activation of your liking like ReLU or tanh
    ##output layer:
    Zo = np.dot(..., ...) + ... #complete the  ...
    Ao = softmax(...) #complete the  ...

    ##we make a cache, that is a 'memory' so that later we can go back in our reasoning when we do back propagation
    cache = {"Z1": Z1,
             "A1": A1,
             "Zo": Zo,
             "Ao": Ao}
    
    ##the output is our probabilities. 
    return Ao, cache

##Check!!
#ao , cache = forward_prop(train_scaled.T, parameters)
#print(ao.shape, cache)

(10, 50000) {'Z1': array([[-7.48417134, -9.74897675, -2.69714221, ..., -2.32342256,
         1.51079418, -8.87261142],
       [-5.5108277 , -5.84043228,  8.72033166, ...,  0.1108445 ,
        -6.82427461,  9.65278565],
       [ 2.06974446,  4.18607877, -7.83777494, ..., -6.65157498,
         1.79771686, -4.01848792],
       ...,
       [-1.33849355, -1.4438885 , -3.62665483, ...,  2.36729967,
        -5.42317423,  5.01208294],
       [ 3.04731579, -9.94127643, 12.97643931, ..., -0.99049326,
         3.81720559,  0.64652026],
       [ 3.18008206,  2.99445515,  2.72347061, ..., -4.19819751,
        -7.41945631,  7.73275125]]), 'A1': array([[-0.99999937, -0.99999999, -0.99095614, ..., -0.98099863,
         0.90707989, -0.99999996],
       [-0.99996731, -0.99998309,  0.99999995, ...,  0.11039276,
        -0.99999764,  0.99999999],
       [ 0.96863765,  0.99953767, -0.99999969, ..., -0.99999666,
         0.94656906, -0.99935364],
       ...,
       [-0.87130994, -0.89447778, -0.99858535, ..

##The objective function (NLL)

First we need to define our cost function, the one that will give us the overall error we made in our predictions. We are going to use the negative log likelihood (NLL). Refer to slides 19 and 20 as a guide (I will change the notation slightly because I will use matrices.)

Let $Y$ be the true labels of our dataset and $A_o$ our estimation from the output layer in the MLP. For our $n$ samples we define the NLL as:

$\mathcal{L} = -\frac{1}{n}\sum_i^n Y_i * \ln(A_{o(i)})$

In [ ]:

def nll_cost(Ao, Y, parameters):

    n = Y.shape[0]

    logprobs = np.multiply(np.log(Ao), Y.T)
    cost =  (-1./n)*np.sum(logprobs)
    cost = np.squeeze(cost)     

    return cost

##nll_cost(ao, one_hot(train_y), parameters) # <- check!

19.030600548583934

##Backward propagation

Refer to the slides from 21 onwards (gradients for multiclass). Again, the notation changed a little, but we are doing what the ppt says.

Now we are going to revise our model from the output back to the input using the gradient of our loss (use the derivative chain rule!). 

We start with the cost function (we use matrix notation, so we need the term $\frac{1}{n}$, for our $n$ samples):

$\mathcal{L} = -\frac{1}{n}\sum_i^n Y_i * \ln(A_{o(i)})$

and we compute the derivative $dZ_o = \frac{d\mathcal{L}}{dZ_o}$: 

$dZ_o = \frac{\partial \mathcal{L}}{\partial A_o} \frac{\partial \mathcal{A_o}}{\partial Z_o} = A_o - Y$

We do the same for $dW_* = \frac{d\mathcal{L}}{dW_*}$, and $db_* = \frac{d\mathcal{L}}{db_*}$

Compute these derivatives yourself using the chain rule (use the slides 21 and 22 as reference)

In [ ]:
def back_prop(parameters, cache, X, Y):
    
    ##Amount of examples
    n = X.shape[0]

    ##Load current parameter weights
    W_xh = parameters["W_xh"]
    Wo = parameters["Wo"]
    
    ##Load the activation information of each layer
    A1 = cache["A1"]
    Ao = cache["Ao"]
    
    ##Let's compute the derivatives! Note we are going backwards, from the output layer to the hidden layer
    dZo= Ao - Y.T
    #dim of dWo should be (10, 200)
    dWo = (1./n)*np.dot(..., ...) #complete the  ...
    #dim of dbo should be (10, 1)
    dbo = (1./n)*np.sum(..., axis=1, keepdims=True) #complete the  ...
    #dim of dZ1 should be (200, 50000)
    dZ1 = np.dot(dWo.T, dZo) * derivative_of_your_activation(...) #complete the information wrt the activation that you chose
    #dim of dW_xh should be (200, 784)
    dW_xh = (1./n)*np.dot(...) #complete the  ...
    #dim of db_h should be (200, 1)
    db_h = (1./n)*np.sum(..., axis=1, keepdims=True) #complete the  ...

    ##Save the gradients in a dictionary to update the parameters
    grads = {"dW_xh": dW_xh,
             "db_h": db_h,
             "dWo": dWo,
             "dbo": dbo}

    return grads

#check!
##grads = back_prop(parameters, cache, train_scaled, one_hot(train_y))


##Update 

Now we have saved all our gradient information from the backward pass. It is time to update our parameters. This means that we will chage their values in the direction that minimizes the cost.

We have different dtrategies to update our parameters, but we will use the simple gradient descent algorithm. The parameter updates will be given by:

$W := W - \alpha . dW$

$b := b - \alpha . db$

The intuition is that we update each value in our parameters 'a little' (we control how much through the learning rate lr = $\alpha$) in the direction that minimizes the gradient.

In [ ]:
def update(parameters, grads, lr):
    
    #Load your current parameters from the forward prop step
    W_xh = parameters["W_xh"]
    b_h = parameters["b_h"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]
    
    #Load the derivatives of your parameters from the backward step
    dW_xh = grads["dW_xh"]
    db_h = grads["db_h"]
    dWo = grads["dWo"]
    dbo = grads["dbo"]

    #Update your parameters using the gradient descent algorithm
    W_xh = W_xh - lr * dW_xh.T
    b_h = b_h - lr * db_h.T
    Wo = ... #complete the  ...
    bo = ... #complete the  ...

    #Store your new parameters
    parameters = {"W_xh": W_xh,
                  "b_h": b_h,
                  "Wo": Wo,
                  "bo": bo}

    return parameters
update(parameters, grads, lr)

{'W_xh': array([[-1.55774025, -0.55499937, -0.16835097, ..., -0.2966716 ,
         -0.53903288, -0.69332885],
        [ 1.2696321 ,  1.50435164, -1.8259483 , ...,  0.08249186,
         -1.24478497, -1.1206462 ],
        [-2.19710091, -1.1843035 ,  0.09566839, ..., -0.53427223,
          0.45166789,  0.969216  ],
        ...,
        [-0.84453958, -1.11615351, -1.85565737, ..., -1.1208382 ,
          1.41601621, -1.29885875],
        [ 0.16231389, -1.37521115, -1.33226617, ...,  0.00365365,
          0.02775971,  0.12394979],
        [-0.39244818,  0.42103832, -0.16005907, ..., -0.19209251,
         -1.17253601,  1.07099013]]),
 'Wo': array([[-0.11788564, -1.58551684, -0.54166225, ..., -0.20386236,
         -0.98964773, -1.31649244],
        [-1.34269869, -1.01850265,  0.56986165, ..., -2.59670563,
          0.8970028 , -0.36322349],
        [ 1.66114858,  2.15758466, -0.56842417, ...,  0.23124303,
         -0.67989288,  0.98769849],
        ...,
        [-0.1401697 , -0.18218567,  2.21

##Almost done!

Before putting everything together for training, let's define one more function. This will compute the accuracy of our predictions during training, and this way we can control how the model is working during training. It is not mandatory, but it is useful and later you can use it for testing.

In [ ]:
def predict(prediction, labels):
    #get the label with the maximum probability in our estimation
    predictions = np.argmax(prediction, axis=0) 
    #check our estimation wrt the true label
    new = np.array([labels[i] == predictions[i] for i,_ in enumerate(labels)], dtype=np.bool)
    #compute the accuracy
    acc = np.count_nonzero(new*1)/labels.shape[0]
    
    return acc * 100

## All together: the MLP training

Congratulations, you survived! Let's put all together now :)

In [ ]:
def MLP_model_train(X, train_y, H, O, D, lr, num_epochs, print_cost):
    
    ##initialize the parameters 
    parameters = initp(D, H, O)

    ##Load your initialized values
    W_xh = parameters["W_xh"]
    b_h = parameters["b_h"]
    Wo = parameters["Wo"]
    bo = parameters["bo"]

    cost_per_epoch = []
    #Train!
    for i in range(0, num_epochs):
        #forward prop
        Ao, cache = forward_prop(...) #complete the  ...

        #compute the cost and save it
        cost = nll_cost(..., one_hot(train_y), ...) #complete the  ...
        cost_per_epoch.append(cost)

        #compute the ccuracy just for reference
        acc = predict(...) #complete the  ...

        #backpropagate!
        grads = back_prop(...,...,..., one_hot(train_y)) #complete the  ...

        #update!
        parameters = update(..., ..., ...) #complete the  ...

        ##print the results every 10 epochs
        if print_cost and i % 10 == 0:
            print ("Cost after iteration %i: %f with accuracy of %f" %(i, cost, acc))
            
    print('Training ended.')
    return parameters, cost_per_epoch

MLP_model_train(train_scaled.T, train_y, h, C, x_dim, lr, num_epochs, print_cost=True)

Cost after iteration 0: 18.884961 with accuracy of 15.932000
Cost after iteration 10: 18.510577 with accuracy of 16.276000


KeyboardInterrupt: ignored

**How do we know if the training is running fine?**


1.   The cost should decrease in each epoch
2.   The accuracy should increase in each epoch

**Does it take long to train and get a good accuracy?**
Yes, it does, so patience :)

When do I stop training? When the cost does not change anymore and starts fluctuating around a certain value (it means it converged)

## Now it is your turn!

What to do next: 

1. Check how much time it took you to train your num_epochs.
1.   Plot the training cost curve using the cost_per_epoch returned from your MLP_model_train() function.
2.   Define a function to test. I will give you the outline, but try to code it yourself. Report the final test accuracy.

Done!



In [ ]:
#This one is a pseudo-code, make sure it works :)
def test(parameters, test_im, test_labels):

  ##Forward prop your new images using your trained parameters
  a_test, cache = forward_prop(test_im, parameters)
  ##Check the accuracy
  acc = predict(....)
  ##compute the cost function of the testing
  cost = nll_cost(....)
  return "Cost for test dataset: %f with accuracy of %f" %(cost, acc)



**Can you do more?**

Sure, have fun. If you want, you can try different number of neurons, learning rates, define a stopping criteria for the training (stop training if the cost converged), define a function to measure the time it takes to train, define a function to plot some examples of images and the labels that the model gives, etc.